### Xây dựng hàm mất mát

Giả sử rằng số _users_ là \\(N\\), số _items_ là \\(M\\), _utility maxtrix_ được mô tả bởi ma trận \\(\mathbf{Y}\\). Thành phần ở hàng thứ \\(m\\), cột thứ \\(n\\) của \\(\mathbf{Y}\\) là _mức độ quan tâm_ (ở đây là số sao đã _rate_) của _user_ thứ \\(n\\) lên sản phẩm thứ \\(m\\) mà hệ thống đã thu thập được. Ma trận \\(Y\\) bị khuyết rất nhiều thành phần tương ứng với các giá trị mà hệ thống cần dự đoán. Thêm nữa, gọi \\(\mathbf{R}\\) là ma trận _rated or not_ thể hiện việc một _user_ đã _rate_ một _item_ hay chưa. Cụ thể, \\(r\_{ij}\\) bằng 1 nếu _item_ thứ \\(i\\) đã được _rated_ bởi _user_ thứ \\(j\\), bằng 0 trong trường hợp còn lại. 



**Mô hình tuyến tính:**

Giả sử rằng ta có thể tìm được một mô hình cho mỗi _user_, minh hoạ bởi vector hệ số \\(\mathbf{w}\_i\\) sao cho _mức độ quan tâm_ của một _user_ tới một _item_ có thể tính được bằng một hàm tuyến tính:

\\[
y_{mn} = \mathbf{x}_m ^T\mathbf{w}_n + b ~~~~(1)
\\]

(_Chúng ta vẫn thường quen nhìn \\(\mathbf{w}^T\mathbf{x}\\) trong các mô hình tuyến tính trước đây, ở đây tôi đảo lại để thứ tự của chỉ số \\(m, n\\) thuận theo thứ tự khi xác định một phần tử của ma trận là (hàng, cột). Hai biểu thức là như nhau vì tích vô hướng của hai vector là một số vô hướng._)

Một lần nữa, để cho các phép tính đơn giản hơn, chúng ta sẽ lại dùng đến [Bias trick](link), tức thêm 1 thành phần bằng 1 nữa vào mỗi feature vector và ghép \\(b\\) vào vector hệ số \\(\mathbf{w}\\). Lúc này, ta sẽ viết lại biểu thức \\(1\\) dưới dạng:

\\[
y_{mn} = \mathbf{x}_m^T\mathbf{w}_n ~~~~(2)
\\]

_Chú ý rằng cả vector hệ số \\(\mathbf{w}\\) và feature vector \\(\mathbf{x}\\) trong \\((2)\\) là khác với chúng trong \\((1)\\) một chút_. 

Xét một _user_ thứ \\(n\\) bất kỳ, nếu ta coi training set là tập hợp các thành phần đã được _điền_ của \\(\mathbf{y}_n\\), ta có thể xây dựng hàm mất mát (của \\(\mathbf{w}_n\\)) tương tự như [Linear Regression] như sau: 

\\[
\mathcal{L}_n = \frac{1}{2} \sum_{m~:~ r_{mn} = 1}(\mathbf{x}_m^T \mathbf{w}_n - y_{mn})^2 + \frac{\lambda}{2} ~~~~~~ (2) \|\|\mathbf{w}_n\|\|_2^2 
\\]

Trong đó, thành phần thứ hai là [regularization term](link) và \\(\lambda\\) là một tham số dương. Trong thực hành, trung bình cộng của _lỗi_ thường được dùng, và _mất mát_ \\(\mathcal{L}\_n\\) được viết lại thành: 

\\[
\mathcal{L}\_n = \frac{1}{2s_n} \sum_{m~:~ r_{mn} = 1}(\mathbf{x}\_m^T \mathbf{w}\_n - y_{mn})^2 + \frac{\lambda}{2s_n} \|\|\mathbf{w}_n\|\|_2^2 
\\]

Trong đó \\(s_n\\) là số lượng các _items_ mà _user_ thứ \\(n\\) đã _rated_. Nói cách khác: 
\\[
s_n = \sum_{m=1}^M r_{mn},
\\]
là tổng các phần tử trên cột thứ \\(n\\) của ma trận _rated or not_ \\(\mathbf{R}\\).

Các bạn đã thấy _loss function_ này _quen_ chưa? 

Vì biểu thức _loss function_ chỉ phụ thuộc vào các _items_ đã được rated, ta có thể rút gọn nó bằng cách đặt \\(\hat{\mathbf{y}}_n\\) là _sub vector_ của \\(\mathbf{y}\\) được xây dựng bằng cách _trích_ các thành phần _khác dấu ?_ ở cột thứ \\(n\\), tức đã được _rated_ bởi _user_ thứ \\(n\\) trong Utility Matrix \\(\mathbf{Y}\\). Đồng thời, đặt \\(\hat{\mathbf{X}}_n\\) là _sub matrix_ của ma trận feature \\(\mathbf{X}\\), được tạo bằng cách _trích_ các cột tương ứng với các _items_ đã được _rated_ bởi _user_ thứ \\(n\\). (_Xem ví dụ phía dưới để hiểu rõ hơn_). Khi đó, biểu thức hàm mất mát của mô hình cho _user_ thứ \\(n\\) được viết gọn thành: 

\\[
\mathcal{L}_n = \frac{1}{2s_n} \|\|\hat{\mathbf{X}}_n^T\mathbf{w}_n - \hat{\mathbf{y}}_n\|\|_2^2 + \frac{\lambda}{2s_n} \|\|\mathbf{w}_n\|\|_2^2
\\]

Đây _chính xác_ là hàm mất mát của [Linear Regression] với regularization. Lời giải có thể trực tiếp thông qua pseudo inverse hoặc Stochastic Gradient Descent (SGD), hoặc Mini-batch GD. Dù bằng cách nào, ta cũng cần tính đạo hàm của hàm mất mát theo \\(\mathbf{w}_n\\):
\\[
\nabla_{\mathbf{w}_n}\mathcal{L}_n = \frac{1}{s_n} \hat{\mathbf{X}}_n(\hat{\mathbf{X}}_n^T\mathbf{w}_n - \hat{\mathbf{y}}_n) + \frac{\lambda}{s_n} \mathbf{w}_n = \frac{1}{s_n} (\hat{\mathbf{X}}_n \hat{\mathbf{X}}_n^T + \lambda \mathbf{I})\mathbf{w}_n - \hat{\mathbf{X}}_n\hat{\mathbf{y}}_n
\\]


Với các bài toán nhỏ, nghiệm _chính xác_ có thể tìm được bằng cách giải phương trình đạo hàm của hàm mất mát theo \\(\mathbf{w}_n\\) bằng 0. Như một bài tập nhỏ, bạn sẽ tìm được:
\\[
\mathbf{w}_n = (\hat{\mathbf{X}}_n \hat{\mathbf{X}}_n^T + \lambda \mathbf{I})^{-1}\hat{\mathbf{X}}_n\hat{\mathbf{y}}_n ~~~~(3)
\\]

trong đó \\(\mathbf{I}\\) là ma trận đơn vị. 

Với các bài toán lớn, tôi chọn cách thứ hai (Mini-batch GD) vì nó có khả năng làm việc với các bài toán _large scale_ với rất nhiều _users_ và _items_. Khi đó, công thức cập nhật cho Mini-batch GD có dạng: 
\\[
\mathbf{w}_n := \mathbf{w}_n - \eta \left(\frac{1}{s_n} (\hat{\mathbf{X}}_n \hat{\mathbf{X}}_n^T + \lambda \mathbf{I})\mathbf{w}_n - \hat{\mathbf{X}}_n\hat{\mathbf{y}}_n\right) ~~~~~ (4)
\\]


Nếu vẫn có điểm chưa hiểu, bạn đọc có thể xem ví dụ nhỏ dưới đây:

### Ví dụ về hàm mất mát cho user E 

Quay trở lại với ví dụ trong hình 2, _feature matrix_ (đã thêm 1 vào hàng cuối cùng theo bias trick) cho các _items_ (mỗi cột tương ứng với một _item_) là: 
\\[
\mathbf{X} = 
\left[
\begin{matrix}
0.99 & 0.91 & 0.95 & 0.01 & 0.03 \\\
0.02 & 0.11 & 0.05 &0.99 & 0.98 \\\
1 & 1 & 1 & 1 & 1
\end{matrix}
\right]
\\]

Xét trường hợp của _user E_ với \\(n = 5\\), \\(\mathbf{y}_5 = [5, ?, ?, 4, ?]^T \Rightarrow \mathbf{r}_5 = [1, 0, 0, 1, 0]^T\\). Vì _E_ mới chỉ _rated_ cho _items_ thứ nhất và thứ tư nên \\(s_5 = 2\\). Hơn nữa:
\\[
\hat{\mathbf{X}}_5 = 
\left[
\begin{matrix}
0.99 &  0.01 \\\
0.02 & 0.99  \\\
1 & 1 
\end{matrix}
\right],
\hat{\mathbf{y}}_5 = \left[
\begin{matrix}
5 \\\
4 
\end{matrix}
\right]
\\]

Khi đó, hàm mất mát cho hệ số tương ứng với _user E_ là: 
\\[
\mathcal{L}_5 = \frac{1}{4} \|\|
\left[
\begin{matrix}
0.99 &  0.02 & 1 \\\
0.01 & 0.99  & 1
\end{matrix}
\right]\mathbf{w}_5 - 
\left[
\begin{matrix}
5 \\\
4
\end{matrix}
\right]
\|\|_2^2 + \frac{\lambda}{4} \|\|\mathbf{w}_5\|\|_2^2
\\]

Và nghiệm theo công thức \\((3)\\) có thể tìm được như sau:

In [9]:
import numpy as np
Xn = np.array([[0.99, 0.01], [0.02, 0.99], [1, 1]])
yn = np.array([[1], [4]])
lam = 0.001 
I = np.eye(3)

wn = np.linalg.inv(Xn.dot(Xn.T) + lam*I).dot(Xn.dot(yn))
print wn

X = np.array([[0.99, 0.91, 0.95, 0.01, 0.03],\
              [0.02, 0.11, 0.05, 0.99, 0.98],\
              [1, 1, 1, 1, 1]])
print X.T.dot(wn)



[[-0.71446013]
 [ 2.36770762]
 [ 1.66070736]]
[[ 1.00074599]
 [ 1.27099648]
 [ 1.10035562]
 [ 3.99759331]
 [ 3.95962703]]


<a name="collaborative-filtering"></a>

## Collaborative Filtering


<a name="tai-lieu-tham-khao"></a>

## Tài liệu tham khảo

[1] Chương 9

[2] Recommendation systems - Machine Learning - Andrew Ng